# Tools
---

I want to collect a series of tools to aid with documenting and carrying out all of my protocols

# Shear Flow Assay (On/Off Rates)
---
1. Set up chips/lanes with populations
2. Count cells
3. Pool Cells
4. Run assay
5. Take notes
6. Pre-process images
7. Annotate
8. Analyze
9. Visualize
10. Report

## Example:

10/28/21
On Rates
General: Comparing binding of ICAM across Lag Binder series
Chip Setup:

Chip 1:
LN1: Jurkat Lag16ICAM Lag17ICAM
LN2: Jurkat Lag16ICAM Lag17ICAM
LN3: Jurkat Lag16ICAM Lag42ICAM
LN4: Jurkat Lag16ICAM Lag42ICAM
LN5: Jurkat Lag16ICAM Lag18ICAM
LN6: Jurkat Lag16ICAM Lag18ICAM

* I want to be able to specify:
    + Population to test (1 cell, 2 cell, 3 cell)
    + How many runs (duplicate, triplicate)
* Program then prompts for **cell concentration and volumes**
    + calculate how many runs you can get at desired target concentration

In [1]:
import math

In [ ]:
exp1 = ["jurkat", "lag16ic", "lag17ic"]
exp2 = ["jurkat", "lag16ic", "lag42ic"]
exp3 = ["jurkat", "lag16ic", "lag18ic"]

experiments = exp1 + exp2 + exp3
counts = Counter(experiments)
counts

In [ ]:
e = []
a = ["x", "y", "z"]
b = ["x", "y"]
c = ["w"]

e.append(a)
e.append(b)
e.append(c)
e

In [ ]:
flat_exps = [c for exp in e for c in exp]
flat_exps

In [ ]:
experiments

In [2]:
adding_new_exp = True
experiments = [] # How to fix this anti pattern

# Need better input UI to avoid typos and shit
# should only be able to add from a bank of validated cell names
while adding_new_exp:
    yes_no = input("Add new exp? [Y/N]")
    if yes_no.lower() == "n":
        adding_new_exp = False
        break
    
    cells = input("Enter cell populations separated by spaces:").split(" ")
    replicates = int(input("Replicates? [1-3]"))
    for i in range(replicates):
        experiments.append(cells)

Add new exp? [Y/N]y
Enter cell populations separated by spaces:jurkat lag16ic_tether lag16ic
Replicates? [1-3]2
Add new exp? [Y/N]y
Enter cell populations separated by spaces:jurkat ncam
Replicates? [1-3]1
Add new exp? [Y/N]n


In [3]:
experiments

[['jurkat', 'lag16ic_tether', 'lag16ic'],
 ['jurkat', 'lag16ic_tether', 'lag16ic'],
 ['jurkat', 'ncam']]

In [4]:
from collections import Counter
flat_exps = [cell 
             for exp in experiments
                for cell in exp]
cell_counts = Counter(flat_exps)
cell_counts

Counter({'jurkat': 3, 'lag16ic_tether': 2, 'lag16ic': 2, 'ncam': 1})

In [ ]:
cell_set = set(flat_exps)
cell_set

target_conc = 2e6 # cells/ml
inj_vol = 200 # ul
single_cell_inj_vol = inj_vol/3 # how much volume of a single cell suspension to add to a pool for an injection

# This number is based on initial protocol of mixing cells 1:1:1 at 2e6 and injecting 0.2ml
# 2e6 cells/ml * 0.2ml = 400,000 cells / 3 cell types => 133,333 cells of one type
single_cell_count = 133333.333



default = True # we should just stick to the default 2e6 concentration unless special circumstances
for cell in cell_set:
    print(f"{cell}: ")
    c1 = float(input("Concentration? [cells/ml] "))
    v1 = int(input("In what volume? [ul] "))
    if not default:
        c2 = float(input("Target concentration? [cells/ml]"))
    else:
        c2 = target_conc
    
    # Final Volume at concentration
    v2 = (c1/c2)*v1
    injections = math.floor(v2/single_cell_inj_vol)
    if injections < cell_counts[cell]:
        print("Not enough volume at target concentration to provide for all experiments")
    print(f"{v2}ul gives {injections} possible injections.")

jurkat: 
Concentration? [cells/ml] 2.4e6
In what volume? [ul] 500
600.0ul gives 9 possible injections.
lag16ic: 
Concentration? [cells/ml] 1.3e6
In what volume? [ul] 500
325.0ul gives 4 possible injections.
ncam: 


In [ ]:
def reconcentrate(name, c1, v1, c2):
    v2 = (c1/c2)*v1

    print(f"Cell {name}:")
    if v2 <= v1: # RESUSPEND
    print(f"* RESUSPEND: @ {v2}ul")
    return v2
    elif v2 > v1: # TOP_OFF
        top_off = v2 - v1
    print(f"* TOP OFF: with {top_off}ul to a total of {v2}ul")
        return v2
    else:
        print("This shouldnt occur?!")

**New experiment? (Y/N)**

**Enter cell populations separated by spaces**

**Replicates? (1-3)**

**


In [ ]:



# Test input with TOP_OFF, RESUSPEND, and LESS_THAN_INJ conditions
#cells = [(2.1e6, 500), (1.7e6, 500), (2.3e5, 500)]
less_than_one_chip_test = [2.1e6, 1.7e6, 2.3e5]
one_chip = [2.1e6, 1.7e6, 1.5e6]
def pool_cells(cells, target_conc):
    initial_vol = 500 # we resuspend our cells in 500ul to count
    inj_vol = 200 # this is the volume we inject for on rates

    lanes_per_chip = 6
    chip_vol = lanes_per_chip * inj_vol
    margin = inj_vol/2
    
    #print(f"Input: {cells}")
    new_cells = [reconcentrate(i+1,c,initial_vol,target_conc) for i,c in enumerate(cells)]

    limiting_vol = min(new_cells)
    mixed_vol = 3 * limiting_vol # eventually replace 3 with how many cell types being run
    runs_possible = math.floor(mixed_vol/inj_vol)


    print(f"Mixed volume is: {mixed_vol}ul")
    if runs_possible < lanes_per_chip + margin:
        print("<!***!>")
        print(f"WARNING: cannot run a full chip with this volume.")
    else:
        pass
    print(f"Runs possible: {runs_possible}")
    print(f">>> Combine {limiting_vol}ul of all cells together.")
    #print(new_cells)
    
pool_cells(one_chip, 2e6)
